In [ ]:
import pandas as pd
import numpy as np

from constants import *
from emissions_parser import emissions
from concs_pulse_decay import pulse_decay_runner
from beam_carbon.beam import BEAMCarbon
beam = BEAMCarbon()
from radiative_forcing import calc_radiative_forcing
from heat_diffusion import continuous_diffusion_model


In [16]:
!pip install beam_carbon

  Could not find a version that satisfies the requirement beam_carbon (from versions: )
No matching distribution found for beam_carbon


### Constants

In [15]:
#Constants used in SimMod modules

TEMP_0 = 10.                #Mean Earth surface temp (degrees C)
C_TO_CO2 = 3.67
PGC_TO_MOL = 1e15/12.
MOLES_IN_ATMOSPHERE = 1.8 * 10.**20.
GRAMS_PER_MOLE_CO2 = 44.01
GRAMS_PER_MOLE_CH4 = 16.04
GRAMS_PER_MOLE_N2O = 44.01
CO2_PER_TON_CH4 = 1.5       #Tons CO2 produced when ton CH4 decays
CH4_EFOLD = 10.
N2O_EFOLD = 114.
CO2_PPM_1750 = 277.01467
CH4_PPB_1750 = 721.89411
N2O_PPB_1750 = 272.95961

CH4_IND_FORCING_SCALAR = 1. #0.970 / 0.641 #Based on IPCC AR5 WG1 Chapter 8 Supp Mats table 8.SM.6. 
#Accounts for indirect CH4 forcing due to tropospheric ozone and stratospheric water vapor. Assumed to scale linearly with atmospheric CH4.


  Could not find a version that satisfies the requirement emissions_parser (from versions: )
No matching distribution found for emissions_parser


### Emissions Parser

In [ ]:
def emissions(run_start_year, run_end_year, dt, rcp, add_start = 0, 
              add_end = 0, c_add = 0, ch4_add = 0, n2o_add = 0):
    """
    Take annual emissions from a RCP scenario and return
    emissions by specified start date, end date, and time step
    """
    run_years = run_end_year - run_start_year + 1 
    rcp_emissions = pd.read_csv('emissions/rcp_'+rcp+'_data.csv', sep=',')
    historic_emissions = pd.read_csv('emissions/historical_ghgs.csv', sep=',')
    emissions = rcp_emissions.append(historic_emissions, ignore_index=True)
    emissions.sort(columns='year', inplace=True)
    emissions.reset_index(inplace=True)
    if add_start > 0:
        emissions.loc[(
            (emissions['year'] >= add_start) & (emissions['year'] <= add_end), 
            'c_emissions_pg')] = emissions['c_emissions_pg'] + c_add
        emissions.loc[(
            (emissions['year'] >= add_start) & (emissions['year'] <= add_end), 
            'ch4_emissions_tg')] = emissions['ch4_emissions_tg'] + ch4_add
        emissions.loc[(
            (emissions['year'] >= add_start) & (emissions['year'] <= add_end), 
            'n2o_emissions_tg')] = emissions['n2o_emissions_tg'] + n2o_add
    
    total_years = emissions.shape[0]
    subset = emissions[int(run_start_year - 1765):int(run_end_year - 1765 + 1)]
    subset.reset_index(inplace=True)
    date = np.array([np.arange(0, run_years, dt)]).T
    columns = ['date']
    df = pd.DataFrame(date, columns=columns)
    steps = df.shape[0]

    for t in range(0,int(steps)):
        df.ix[t, 'year'] = subset['year'][int(df['date'][t])]
        df.ix[t, 'co2_pg'] = subset['c_emissions_pg'][int(df['date'][t])] * dt * C_TO_CO2
        df.ix[t, 'ch4_tg'] = subset['ch4_emissions_tg'][int(df['date'][t])] * dt
        df.ix[t, 'n2o_tg'] = subset['n2o_emissions_tg'][int(df['date'][t])] * dt
        df.ix[t, 'hist_forcing_wm2'] = subset['hist_forcing_wm2'][int(df['date'][t])]
        df.ix[t, 'co2_forcing_rcp'] = subset['co2_forcing_wm2'][int(df['date'][t])]
        df.ix[t, 'ch4_forcing_rcp'] = subset['ch4_forcing_wm2'][int(df['date'][t])]
        df.ix[t, 'n2o_forcing_rcp'] = subset['n2o_forcing_wm2'][int(df['date'][t])]
        df.ix[t, 'total_forcing_rcp'] = subset['total_forcing_wm2'][int(df['date'][t])]
        df.ix[t, 'rcp_co2_ppm'] = subset['co2_concentration_ppm'][int(df['date'][t])]
        df.ix[t, 'rcp_ch4_ppb'] = subset['ch4_concentration_ppb'][int(df['date'][t])]
        df.ix[t, 'rcp_n2o_ppb'] = subset['n2o_concentration_ppb'][int(df['date'][t])]
    
    df['rcp_co2_ppm'].fillna(CO2_PPM_1750, inplace=True)
    df['rcp_ch4_ppb'].fillna(CH4_PPB_1750, inplace=True)
    df['rcp_n2o_ppb'].fillna(N2O_PPB_1750, inplace=True)
    
    return df

### Pulse Decay

In [ ]:
def pulse_decay_runner(run_years, dt, emissions):
    """
    Simple pulse response model adapted from Myhrvold and Caldeira (2012)
    adapted from Joos et al (1996)
    """
    df = emissions
    co2_0 = df['rcp_co2_ppm'][0] 
    ch4_0 = df['rcp_ch4_ppb'][0] 
    n2o_0 = df['rcp_n2o_ppb'][0]

    run_year = 0
    df['co2_pg_atm'] = 0
    df['ch4_tg_atm'] = 0
    df['n2o_tg_atm'] = 0
    df['ch4_co2_decay_marginal'] = 0
    while run_year < run_years:
        df['ch4_step'] = (
            df['ch4_tg'][int(run_year / dt)] * 
            np.exp(-(df['date'] - run_year) / CH4_EFOLD)
        )
        df['ch4_step'][0:int(run_year / dt)] = 0
        
        df['ch4_co2_decay'] = (df['ch4_tg'][int(run_year / dt)] - df['ch4_step']) * CO2_PER_TON_CH4 / 10**3
        #convert from TgC to PgC

        df['ch4_co2_decay'][0:int(run_year / dt)] = 0
        df['ch4_co2_decay_marginal'] += df['ch4_co2_decay'].diff(-1) * -1
        df['ch4_co2_decay_marginal'][-1:] = 0

        df['co2_step'] = (
            (df['co2_pg'][int(run_year / dt)] + df['ch4_co2_decay_marginal'][int(run_year / dt)]) * 
            (0.217 + 0.259 * np.exp(-(df['date'] - run_year) / 172.9) + 
            0.338 * np.exp(-(df['date'] - run_year) / 18.51) + 
            0.186 * np.exp(-(df['date'] - run_year) / 1.186))
        )

        df['n2o_step'] = (
            df['n2o_tg'][int(run_year / dt)] * 
            np.exp(-(df['date'] - run_year) / N2O_EFOLD)
        )
        df['co2_step'][0:int(run_year / dt)] = 0
        df['n2o_step'][0:int(run_year / dt)] = 0

        df['co2_pg_atm'] += df['co2_step']
        df['ch4_tg_atm'] += df['ch4_step']
        df['n2o_tg_atm'] += df['n2o_step']

        run_year += dt

    df['co2_ppm']  = (
        co2_0 + (df['co2_pg_atm'] * 10.**15. / GRAMS_PER_MOLE_CO2) / 
        MOLES_IN_ATMOSPHERE * 10.**6.
    )

    df['ch4_ppb']  = (
        ch4_0 + (df['ch4_tg_atm'] * 10.**12. / GRAMS_PER_MOLE_CH4) / 
        MOLES_IN_ATMOSPHERE * 10.**9.
    )

    df['n2o_ppb']  = (
        n2o_0 + (df['n2o_tg_atm'] * 10.**12. / GRAMS_PER_MOLE_N2O) / 
        MOLES_IN_ATMOSPHERE * 10.**9.
    )
    df = df.drop(['ch4_step', 'co2_step', 'n2o_step', 'ch4_co2_decay'], axis=1)
    return df

### Heat Diffusion

In [ ]:
#Model Variables
LAYER_HEIGHT = 100.
TOTAL_HEIGHT = 3700.

#Model Constants
HEAT_CAPACITY = 3985. * 1024.5 # J m^-3 K^-1
KAPPA = 5.5 * 10**-5           #m^2 s^-1
OCEAN_PERCENT = 0.71


def diffeqs(df, dt, fradfor, clim_sens):
    """
    Differential equation for flux down.
    """
    steps = df.shape[0]
    df.ix[0, 'fluxdown'] = (
        (((fradfor - clim_sens * df['tocean']) / HEAT_CAPACITY) * dt)[0]
    )
    df['fluxdown'] = (
        (KAPPA * (df['tocean'].shift(1) - df['tocean']) / LAYER_HEIGHT) * dt
    )
    df.ix[0, 'fluxdown'] = (
        (((fradfor - clim_sens * df['tocean']) / HEAT_CAPACITY) * dt)[0]
    )

    df['dtocean'] = (df['fluxdown'].diff(periods = -1) / LAYER_HEIGHT) * dt

    df.ix[(steps - 1), 'dtocean'] = (
        (df['fluxdown'] / LAYER_HEIGHT * dt)[steps - 1]
    )
    return df


def continuous_diffusion_model(results, run_years, dt, clim_sens):
    """
    Implement the continuous diffusion model
    used in Myhrvold and Cairdira (2011).
    """
    z = np.array([np.arange(0, (TOTAL_HEIGHT + LAYER_HEIGHT), LAYER_HEIGHT)]).T
    columns = ['z']
    df = pd.DataFrame(z, columns=columns)
    df['tocean'] = 0
    df['dtocean'] = 0
    df['fluxdown'] = 0
    fradfor = results['total_forcing'][0]
    df = diffeqs(df, dt, fradfor, clim_sens)

    for t in range(0,int(run_years / dt)):
        fradfor = results['total_forcing'][t]
        df['tocean'] += dt * df['dtocean'] * 365 * 24 * 60 * 60
        df = diffeqs(df, dt, fradfor, clim_sens)
        results.ix[t, 't_os'] = df.ix[0, 'tocean']

    results['t_eq'] = results['total_forcing'] / clim_sens
    results['t_s'] = (
        results['t_os'] * OCEAN_PERCENT + 
        results['t_eq'] * (1 - OCEAN_PERCENT)
    )

    return results

In [14]:

#Model Parameters
run_start_year = 1765.          #Run start year
run_end_year = 2100.            #Inclusive of end year
dt = 1 #/ 100.                  #years
rcp = '8.5'                     #RCP scenario
carbon_model = 'pulse response' #'pulse response', 'box diffusion', or 'BEAM'
normalize_2000_conc = True      #Normalize concentrations to historical year-2000 values
c_sens = 1.25                   #Climate sensativity (T = F / LAMBDA)

#BEAM Model Settings (when relevant)
SUBSTEPS = 100                  #Break each timestep into this many substeps
INIT_MAT = 596.                 #In GtC; 596 = preindustrial; 809 = 2005
INIT_MUP = 713.                 #In GtC; 713 = preindustrial; 725 = 2005
INIT_MLO = 35625.               #In GtC; 35625 = preindustrial; 35641 = 2005

#Ocean Box Diffusion Model Settings (when relevant)
MIXING = 'probable'             #options 'fast', 'slow', or 'probable'
DZ = 100                        #meters - thickness of each layer in the deep ocean


def run_simmod(run_start_year, run_end_year, dt, rcp, c_sens = c_sens, add_start = 0, 
               add_end = 0, c_add = 0, ch4_add = 0, n2o_add = 0):
    """
    Run the various parts of SimMod and export images and CSV files.
    """
    run_years = (run_end_year - run_start_year + 1)
    emission_vals = emissions(run_start_year, run_end_year, dt, rcp, 
                              add_start, add_end, c_add, ch4_add, n2o_add)
    conc = pulse_decay_runner(run_years, dt, emission_vals)

    if carbon_model == 'BEAM':
        beam._initial_carbon = np.array([596., 713., 35625.])
        beam.intervals = SUBSTEPS
        beam.time_step = dt
        beam.emissions = emission_vals['co2_pg'] / C_TO_CO2
        beam_results = pd.melt(beam.run()[0:1])
        conc['co2_ppm'] = beam_results['value'] * PGC_TO_MOL * 1e6 / MOLES_IN_ATMOSPHERE

    if carbon_model == 'box diffusion':
        box_diffusion_results = box_diffusion_model(
            emission_vals, 
            dt, 
            DZ, 
            MIXING
        )
        conc['co2_ppm'] = box_diffusion_results['co2ppm']

    if normalize_2000_conc == True:
        conc['co2_ppm'] = (
            conc['co2_ppm'] - 
            conc.loc[conc['year'] == 2000, 'co2_ppm'].min() +
            emission_vals.loc[emission_vals['year'] == 2000, 'rcp_co2_ppm'].min()
        )
        conc['ch4_ppb'] = (
            conc['ch4_ppb'] - 
            conc.loc[conc['year'] == 2000, 'ch4_ppb'].min() +
            emission_vals.loc[emission_vals['year'] == 2000, 'rcp_ch4_ppb'].min()
        )
        conc['n2o_ppb'] = (
            conc['n2o_ppb'] - 
            conc.loc[conc['year'] == 2000, 'n2o_ppb'].min() +
            emission_vals.loc[emission_vals['year'] == 2000, 'rcp_n2o_ppb'].min()
        )

    forcing = calc_radiative_forcing(conc)
    warming = continuous_diffusion_model(forcing, run_years, dt, c_sens)
    return warming

results = run_simmod(run_start_year, run_end_year, dt, rcp, c_sens)
results.to_csv('results/simmod_run_'+rcp+' '+carbon_model+'.csv')

ModuleNotFoundError: No module named 'emissions_parser'